#Team 7 - Environmental Feature Extraction
This workbook contains the python and SQL code used to extract the environmental features from different data sources and prepare them for use in the next step of our analysis.

In [ ]:
#Install mysql and establish the database connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/

In [ ]:
#Establish connect for Google drive to access files
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Creating the database


In [ ]:
#This only needs to be run once - when you create the database
%sql CREATE DATABASE mlproject;

 * mysql+pymysql://root:***@/


""


###Extracting from Tara Registries Water Column
This section extracts the nitrate levels along with associated sample and station details. Then 2 derived columns are added using details from the other columns in the table.

In [ ]:
#Copy the file to the "safe" directory and change the access to the file
%cp gdrive/MyDrive/TARA_registies_water_column.tab /var/lib/mysql-files/
!chmod 755 /var/lib/mysql-files/TARA_registies_water_column.tab

In [ ]:
#Find MySQL safe directory
%sql SHOW VARIABLES LIKE "secure_file_priv"

 * mysql+pymysql://root:***@/


,Variable_name,Value
0,secure_file_priv,/var/lib/mysql-files/


In [ ]:
#create the table that will hold the data from the Tara Registries Water Column Table
%sql CREATE TABLE mlproject.tara_samples (sample_id varchar(20), station_id varchar(10), date_time_of_event datetime(0), latitude decimal(20,4), longitude decimal(20,4), env_feature varchar(250), depth varchar(20), nitrate_level_1 decimal(20,6), nitrate_level_2 decimal(20,6), nitrate_level_3 decimal(20,6), nitrate_level_4 decimal(20,6), nitrate_level_5 decimal(20,6), nitrate_level_6 decimal(20,6), nitrate_level_7 decimal(20,6), nitrate_level_8 decimal(20,6), nitrate_level_9 decimal(20,6), nitrate_level_10 decimal(20,6), temp_level_1 decimal(20,6), temp_level_2 decimal(20,6), temp_level_3 decimal(20,6), temp_level_4 decimal(20,6), temp_level_5 decimal(20,6), temp_level_6 decimal(20,6), temp_level_7 decimal(20,6), temp_level_8 decimal(20,6), temp_level_9 decimal(20,6), temp_level_10 decimal(20,6), salinity_level_1 decimal(20,6), salinity_level_2 decimal(20,6), salinity_level_3 decimal(20,6), salinity_level_4 decimal(20,6), salinity_level_5 decimal(20,6), salinity_level_6 decimal(20,6), salinity_level_7 decimal(20,6), salinity_level_8 decimal(20,6), salinity_level_9 decimal(20,6), salinity_level_10 decimal(20,6), oxygen_level_1 decimal(20,6), oxygen_level_2 decimal(20,6), oxygen_level_3 decimal(20,6), oxygen_level_4 decimal(20,6), oxygen_level_5 decimal(20,6), oxygen_level_6 decimal(20,6), oxygen_level_7 decimal(20,6), oxygen_level_8 decimal(20,6), oxygen_level_9 decimal(20,6), oxygen_level_10 decimal(20,6));

 * mysql+pymysql://root:***@/


""


In [ ]:
#Remove strict sql mode for the session before starting import to handle empty values for nitrate levels
%sql SET SESSION sql_mode = '';

 * mysql+pymysql://root:***@/


""


In [ ]:
#Load data from file
%sql LOAD DATA INFILE "/var/lib/mysql-files/TARA_registies_water_column.tab" INTO TABLE mlproject.tara_samples (sample_id, @dummy, station_id, @dummy, date_time_of_event, latitude, longitude, env_feature, depth, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, temp_level_1, temp_level_2, temp_level_3, temp_level_4, temp_level_5, temp_level_6, temp_level_7, temp_level_8, temp_level_9, temp_level_10, salinity_level_1, salinity_level_2, salinity_level_3, salinity_level_4, salinity_level_5, salinity_level_6, salinity_level_7, salinity_level_8, salinity_level_9, salinity_level_10, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, @dummy, oxygen_level_1, oxygen_level_2, oxygen_level_3, oxygen_level_4, oxygen_level_5, oxygen_level_6, oxygen_level_7, oxygen_level_8, oxygen_level_9, oxygen_level_10, nitrate_level_1, nitrate_level_2, nitrate_level_3, nitrate_level_4, nitrate_level_5, nitrate_level_6, nitrate_level_7, nitrate_level_8, nitrate_level_9, nitrate_level_10);

 * mysql+pymysql://root:***@/


""


In [ ]:
#Add columns for ocean level and sample month (derived from other attributes)
%sql ALTER TABLE mlproject.tara_samples ADD COLUMN level_code varchar(3), ADD COLUMN sample_month int;

 * mysql+pymysql://root:***@/


""


In [ ]:
#Populate the new columns
%sql UPDATE mlproject.tara_samples ts SET ts.level_code = SUBSTRING(ts.env_feature, 2, 3), ts.sample_month = MONTH(ts.date_time_of_event);

 * mysql+pymysql://root:***@/


""


In [ ]:
#Confirm 34,773 records loaded
%sql SELECT COUNT(*) FROM mlproject.tara_samples;

 * mysql+pymysql://root:***@/


,COUNT(*)
0,34773


###Identify stations of interest
Using the samples that met our criteria for analysis (5kb+ or circular) as a starting point, identify the locations (station, depth combination) that are of interest for our analysis. This list will be used to filter the table loaded above.

In [ ]:
!pip install biopython

     |████████████████████████████████| 2.3 MB 27.7 MB/s 


In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import GC
from Bio.SeqIO.FastaIO import SimpleFastaParser
import pandas as pd
import numpy as np

In [ ]:
# Initialize lists
seq_ids=[]
station_id=[]
depth_class=[]

def convert_station_name(station_list):
    #replace the word Station with TARA_ to facilitate matching with other data set
    for item in station_list:
        item[0] = item[0].replace('Station', 'TARA_')
    #for stations with only 2 digits add a leading 0
    for item in station_list:
        if item[0][-3] == '_':
            item[0] = item[0].replace('_', '_0')
    return station_list


def convert_depth(station_list):
    #modify the depth codes to align with other sources
    for item in station_list:
        if item[1] == 'SUR':
            item[1] = 'SRF'
        if item[1] == 'MXL':
            item[1] = 'MIX'
    return station_list

# Parsing features from viral sequence fasta files
with open('gdrive/MyDrive/GOV2_viral_populations_larger_than_5KB_or_circular.fasta') as file:
    for header,sequence in SimpleFastaParser(file):
        seq_ids.append(header)

station_list = []
for line in seq_ids:
    if len(line)!=0:
        station = line.split('_')[0]
        depth_class = line.split('_')[1]
    if station.startswith('Station'):
        station_list.append([station, depth_class])

station_list = convert_station_name(station_list)
station_list = convert_depth(station_list)


#extract stations from Normalized_Viral_Abundance_ALL_5kb.txt
initial_list = []
station_list2 = []
with open('gdrive/MyDrive/Normalized_Viral_Abundances_ALL_5kb.txt') as file2:
    first_line = file2.readline()

initial_list = first_line.split('\t')
for item in initial_list:
    if item.startswith('Station'):
        station_list2.append(item.split('_'))

station_list2 = convert_station_name(station_list2)
station_list2 = convert_depth(station_list2)

#combine the 2 lists, selecting only unique values
combo_station_list = []

for item in station_list2:
    if item not in combo_station_list:
        combo_station_list.append(item)
for item in station_list:
    if item not in combo_station_list:
        combo_station_list.append(item)

#convert to data frame and save
df = pd.DataFrame(combo_station_list)
df.columns = ['station_id','depth_class']
df.to_csv("/var/lib/mysql-files/station_list.csv", index=False)

###Load Stations of Interest to Database

In [ ]:
%sql CREATE TABLE mlproject.stations_of_interest (station_id varchar(10), depth_class varchar(3));

 * mysql+pymysql://root:***@/


""


In [ ]:
%sql LOAD DATA INFILE "/var/lib/mysql-files/station_list.csv" INTO TABLE mlproject.stations_of_interest FIELDS TERMINATED BY ',' IGNORE 1 LINES;

 * mysql+pymysql://root:***@/


""


In [ ]:
#Confirm 134 records loaded
%sql SELECT COUNT(*) FROM mlproject.stations_of_interest; 

 * mysql+pymysql://root:***@/


,COUNT(*)
0,134


###Convert EBI BioSamples file
In order to get nitrate levels for more of our samples, we exported a file from <a href="https://www.ebi.ac.uk/biosamples/samples?text=TARA">EBI</a>.  The format is JSON; the code below converts the JSON file to csv so it can be imported to the database.

In [ ]:
#import json file to a data frame
df = pd.read_json("gdrive/MyDrive/BioSamples.json")

#extract column containing the characteristics of each sample
char_dict = df['characteristics']
dict_len = len(char_dict)

#find the sample id (from alias or submitter id) and nitrate sensor level
data = []
i = 0
while i < dict_len:
	#extract alias, if present
	alias = char_dict[i].get('alias')
	if alias:
		alias = alias[0]['text']
	#extract submitter id, if present
	submitter_id = char_dict[i].get('Submitter Id')
	if submitter_id:
		submitter_id = submitter_id[0]['text']
	#if the alias or submitter_id start with TARA_, move them to the sample_id field 
	sample_id = "" #default sample id to a blank value
	if alias is not None and alias.startswith('TARA_'):
		sample_id = alias
	elif submitter_id is not None and submitter_id.startswith('TARA'):
		sample_id = submitter_id
	#extract nitrate level, if present
	nitrate_level = char_dict[i].get('nitrate sensor')
	if nitrate_level:
		nitrate_level = float(nitrate_level[0]['text'])
	#extract temperature, if present
	temperature = char_dict[i].get('temperature')
	if temperature:
		temperature = float(temperature[0]['text'])
	#extract salinity level, if present
	salinity_level = char_dict[i].get('Salinity Sensor')
	if salinity_level:
		salinity_level = float(salinity_level[0]['text'])
	#extract oxygen level, if present
	oxygen_level = char_dict[i].get('oxygen sensor')
	if oxygen_level:
		oxygen_level = float(oxygen_level[0]['text'])
	#add valid TARA records to the data list
	if sample_id != "":
		data.append([sample_id,nitrate_level,temperature,salinity_level,oxygen_level])
	i += 1

#remove list items that had None for all 3 attributes
filtered_data = list(filter(lambda item: item != [None, None], data))

#convert to a dataframe
df = pd.DataFrame(filtered_data, columns = ['sample_id', 'nitrate_level', 'temperature', 'salinity_level','oxygen_level'])

df.to_csv("/var/lib/mysql-files/BioSamples.csv", index=False)

###Load BioSamples Data

In [ ]:
%sql CREATE TABLE mlproject.ebi_biosamples (sample_id varchar(20), ebi_nitrate_level decimal(20,6), ebi_temperature decimal(20,6), ebi_salinity_level decimal(20,6), ebi_oxygen_level decimal(20,6));

 * mysql+pymysql://root:***@/


""


In [ ]:
%sql LOAD DATA INFILE "/var/lib/mysql-files/BioSamples.csv" INTO TABLE mlproject.ebi_biosamples FIELDS TERMINATED BY ',' IGNORE 1 LINES;

 * mysql+pymysql://root:***@/


""


In [ ]:
#Confirm 59,004 records loaded
%sql SELECT COUNT(*) FROM mlproject.ebi_biosamples; 

 * mysql+pymysql://root:***@/


,COUNT(*)
0,59004


###Create database view
Pull the sample_id, station_id, latitude, longitude directly, month and level from the tara_samples table. nitrate_level_tara is the nitrate level from tara that corresponds with the level that the sample was taken from:


*   If Level = “SRF”, then use nitrate_level_1: [NO3]- [µmol/l] (at a depth of 10 m, below the surface)
*   If Level = “MIX”, then use nitrate_level_4: [NO3]- [µmol/l] (at the depth of the mixed layer (based on sigma theta))
*   If Level = “DCM”, then use nitrate_level_6: [NO3]- [µmol/l] (at the depth of maximum chlorophyll fluorescence)
*   If Level = “MES”, then use nitrate_level_10: [NO3]- [µmol/l] (at the depth of the nitracline)
*   If Level = “ZZZ”, then use nitrate_level_9: [NO3]- [µmol/l] (at the depth of minimum oxygen concentration)

The view is limited to only those stations and depths that are in the 'stations of interest' table

In [ ]:
%sql CREATE VIEW mlproject.tara_nitrate_levels AS SELECT ts.sample_id, ts.station_id, ts.sample_month, ts.latitude, ts.longitude, ts.level_code, (CASE WHEN ts.level_code = 'SRF' THEN nitrate_level_1 WHEN ts.level_code = 'MIX' THEN nitrate_level_4 WHEN ts.level_code = 'DCM' THEN nitrate_level_6 WHEN ts.level_code = 'MES' THEN nitrate_level_10 WHEN ts.level_code = 'ZZZ' THEN nitrate_level_9 END) AS tara_nitrate_level, ts.depth, (CASE WHEN ts.level_code = 'SRF' THEN temp_level_1 WHEN ts.level_code = 'MIX' THEN temp_level_4 WHEN ts.level_code = 'DCM' THEN temp_level_6 WHEN ts.level_code = 'MES' THEN temp_level_10 WHEN ts.level_code = 'ZZZ' THEN temp_level_9 END) AS tara_temperature, (CASE WHEN ts.level_code = 'SRF' THEN salinity_level_1 WHEN ts.level_code = 'MIX' THEN salinity_level_4 WHEN ts.level_code = 'DCM' THEN salinity_level_6 WHEN ts.level_code = 'MES' THEN salinity_level_10 WHEN ts.level_code = 'ZZZ' THEN salinity_level_9 END) AS tara_salinity_level, (CASE WHEN ts.level_code = 'SRF' THEN oxygen_level_1 WHEN ts.level_code = 'MIX' THEN oxygen_level_4 WHEN ts.level_code = 'DCM' THEN oxygen_level_6 WHEN ts.level_code = 'MES' THEN oxygen_level_10 WHEN ts.level_code = 'ZZZ' THEN oxygen_level_9 END) AS tara_oxygen_level FROM mlproject.tara_samples AS ts INNER JOIN mlproject.stations_of_interest AS si ON ts.station_id = si.station_id AND ts.level_code = si.depth_class;

 * mysql+pymysql://root:***@/


""


###Extract data for Simons CMAP lookup
We still have records without nitrate information so this section extracts the station location information so that we can retrieve data from Darwin on Simons CMAP.

In [ ]:
%sql CREATE VIEW mlproject.nitrate_level_export AS SELECT tara.sample_id, tara.station_id, tara.sample_month, tara.latitude, tara.longitude, tara.level_code, tara.tara_nitrate_level, tara.depth, ebi.ebi_nitrate_level, (CASE WHEN tara.tara_nitrate_level <> 0 THEN tara.tara_nitrate_level WHEN (ebi.ebi_nitrate_level <> 99999 AND ebi.ebi_nitrate_level <> 9999) THEN ebi.ebi_nitrate_level END) AS nitrate_level, tara.tara_temperature, ebi.ebi_temperature, (CASE WHEN tara.tara_temperature <> 0 THEN tara.tara_temperature WHEN ebi.ebi_temperature <> 99999 THEN ebi.ebi_temperature END) AS temperature, tara.tara_salinity_level, ebi.ebi_salinity_level, (CASE WHEN tara.tara_salinity_level <> 0 THEN tara.tara_salinity_level WHEN ebi.ebi_salinity_level <> 99999 THEN ebi.ebi_salinity_level END) AS salinity_level, tara.tara_oxygen_level, ebi.ebi_oxygen_level, (CASE WHEN tara.tara_oxygen_level <> 0 THEN tara.tara_oxygen_level WHEN ebi.ebi_oxygen_level <> 99999 THEN ebi.ebi_oxygen_level END) AS oxygen_level FROM mlproject.tara_nitrate_levels AS tara LEFT JOIN mlproject.ebi_biosamples AS ebi ON tara.sample_id = ebi.sample_id;

 * mysql+pymysql://root:***@/


""


In [ ]:
%sql SELECT 'sample_id', 'sample_month', 'latitude', 'longitude', 'depth' UNION ALL SELECT sample_id, sample_month, latitude, longitude, depth FROM mlproject.nitrate_level_export INTO OUTFILE '/var/lib/mysql-files/darwin_lookup.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\r\n';

 * mysql+pymysql://root:***@/


""


###Extract Darwin data
This section of code accesses the Darwin data in the Simons CMAP via their api. It initially looks for an exact match with our station depth; if there's no match it recalls the api with other values in close proximity. [Warning: Calling the api for each of these records typically takes 3-4 hours.]

In [ ]:
!pip install pycmap

     |████████████████████████████████| 223 kB 30.6 MB/s 
  Created wheel for pycmap: filename=pycmap-0.1.10-py3-none-any.whl size=42438 sha256=f6f90118560b007c994f6e8055201be7e93e7286aa034a93410c0c163ae214e3
  Stored in directory: /root/.cache/pip/wheels/12/94/93/bcf4fe043b8e78ae2f418a7746c9aaf437ba3a1af4324c9746
Successfully built pycmap


In [ ]:
import pycmap
import math

In [ ]:
api = pycmap.API(token='XXXXXXXXXXXXXXXX') #token removed from this shared version for security reasons

In [ ]:
#Extract the catalog for more details on data
catalog = api.get_catalog()
catalog.to_excel('catalog.xlsx')

In [ ]:
#import TARA details
tara_loc = pd.read_csv("/var/lib/mysql-files/darwin_lookup.csv")

result_dict = {}

def call_lookup(tara_month, rounded_lat, rounded_lon, tara_depth):
    query_string = ("SELECT (NO2_darwin_clim + NO3_darwin_clim) AS total_nitrate_level, ALK_darwin_clim, O2_darwin_clim FROM tblDarwin_Nutrient_Climatology WHERE month = " + str(tara_month) + " AND lat = " + str(rounded_lat) + " AND lon = " + str(rounded_lon) + " AND depth = " + str(tara_depth))
    query_result = api.query(query_string)
    return query_result

#Look up each combination of values in the SimonsCMAP and save in a dictionary
for row in tara_loc.itertuples(index=False):
    attempt = 1
    tara_month = row.sample_month
    if '-' in row.depth:
        tara_depth = int(row.depth.split('-')[0])
    else:
        tara_depth = int(row.depth)
    lat_frac, lat_whole = math.modf(row.latitude)
    if 0 <= abs(lat_frac) < 0.5:
        if lat_whole >= 0:
            rounded_lat = lat_whole + 0.25
        else:
            rounded_lat = lat_whole - 0.25
    else:
        if lat_whole >= 0:
            rounded_lat = lat_whole + 0.75
        else:
            rounded_lat = lat_whole - 0.75
    lon_frac, lon_whole = math.modf(row.longitude)
    if 0 <= abs(lon_frac) < 0.5:
        if lat_whole >= 0:
            rounded_lon = lon_whole + 0.25
        else:
            rounded_lon = lon_whole - 0.25
    else:
        if lat_whole >= 0:
            rounded_lon = lon_whole + 0.75
        else:
            rounded_lon = lon_whole - 0.75
    query_result = call_lookup(tara_month, rounded_lat, rounded_lon, tara_depth)
    if not query_result.empty:
        result_dict[row.sample_id] = [tara_month, tara_depth, rounded_lat, rounded_lon, query_result['total_nitrate_level'][0], query_result['ALK_darwin_clim'][0], query_result['O2_darwin_clim'][0]]
    elif query_result.empty:
        while attempt < 5:
            #If the lookup did not work, try rounding to a multiple of 5 (if it isn't already); if it already is, try adding 5
            if tara_depth % 10 == 0 or tara_depth % 10 == 5:
                tara_depth += 5
            else:
                tara_depth = (5 * round(tara_depth/5))
            query_result = call_lookup(tara_month, rounded_lat, rounded_lon, tara_depth)
            if not query_result.empty:
                result_dict[row.sample_id] = [tara_month, tara_depth, rounded_lat, rounded_lon, query_result['total_nitrate_level'][0], query_result['ALK_darwin_clim'][0], query_result['O2_darwin_clim'][0]]
                break
            attempt += 1

#Convert the dictionary to a df and export to csv
nitrate_value_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['tara_month','depth_used','lat_used','lon_used','darwin_nitrate_level','darwin_alkalinity','darwin_oxygen_level'])
nitrate_value_df.to_csv("/var/lib/mysql-files/darwin_nitrate_level.csv")


###Import Darwin data

In [ ]:
%sql CREATE TABLE mlproject.darwin_data (sample_id varchar(20), month_used int, depth_used int, lat_used decimal(10,2), lon_used decimal(10,2), darwin_nitrate_level decimal(20,6), darwin_alkalinity decimal(20,6), darwin_oxygen_level decimal(20,6));

 * mysql+pymysql://root:***@/


""


In [ ]:
%sql LOAD DATA INFILE "/var/lib/mysql-files/darwin_nitrate_level.csv" INTO TABLE mlproject.darwin_data FIELDS TERMINATED BY ',' IGNORE 1 LINES;

 * mysql+pymysql://root:***@/


""


In [ ]:
#Confirm 10,148 records loaded
%sql SELECT COUNT(*) FROM mlproject.darwin_data;

 * mysql+pymysql://root:***@/


,COUNT(*)
0,10039


###Update view and create export for next step

In [ ]:
%sql CREATE VIEW mlproject.nitrate_level_export2 AS SELECT init.sample_id, init.station_id, init.sample_month, init.latitude, init.longitude, init.level_code, init.tara_nitrate_level, init.ebi_nitrate_level, darwin.darwin_nitrate_level, (CASE WHEN init.tara_nitrate_level <> 0 THEN init.tara_nitrate_level WHEN (init.nitrate_level <> 99999 AND init.nitrate_level <> 9999) THEN init.nitrate_level ELSE darwin.darwin_nitrate_level END) AS nitrate_level, init.tara_temperature, init.ebi_temperature, init.temperature, init.tara_salinity_level, init.ebi_salinity_level, init.salinity_level, init.tara_oxygen_level, init.ebi_oxygen_level, darwin.darwin_oxygen_level, (CASE WHEN init.tara_oxygen_level <> 0 THEN init.tara_oxygen_level WHEN init.ebi_oxygen_level <> 99999 THEN init.ebi_oxygen_level ELSE darwin.darwin_oxygen_level END) AS oxygen_level, darwin_alkalinity FROM mlproject.nitrate_level_export AS init LEFT JOIN mlproject.darwin_data AS darwin ON init.sample_id = darwin.sample_id;

 * mysql+pymysql://root:***@/


""


In [ ]:
%sql SELECT 'sample_id', 'station_id', 'latitude', 'longitude', 'level_code', 'tara_nitrate_level', 'ebi_nitrate_level', 'darwin_nitrate_level', 'nitrate_level', 'tara_temperature', 'ebi_temperature', 'temperature','tara_salinity_level', 'ebi_salinity_level', 'salinity_level', 'tara_oxygen_level', 'ebi_oxygen_level', 'darwin_oxygen_level','oxygen_level','darwin_alkalinity' UNION ALL SELECT sample_id, station_id, latitude, longitude, level_code, IFNULL(tara_nitrate_level,""), IFNULL(ebi_nitrate_level,""), IFNULL(darwin_nitrate_level,""), IFNULL(nitrate_level,""), IFNULL(tara_temperature,''), IFNULL(ebi_temperature,''), IFNULL(temperature,''), IFNULL(tara_salinity_level,''), IFNULL(ebi_salinity_level,''), IFNULL(salinity_level,''), IFNULL(tara_oxygen_level,''), IFNULL(ebi_oxygen_level,''), IFNULL(darwin_oxygen_level,''), IFNULL(oxygen_level,''), IFNULL(darwin_alkalinity,'') FROM mlproject.nitrate_level_export2 INTO OUTFILE '/var/lib/mysql-files/nitrate_levels.csv' FIELDS TERMINATED BY ',' LINES TERMINATED BY '\r\n';

 * mysql+pymysql://root:***@/


""
